In [61]:
import pandas as pd
import numpy as np
import os
from os import path
import math

In [66]:
total_frames = 150
desired_frames = 100
path = "/u1/cec383/Fusion_Approach_Pipelines/Chiquet_100_Extra/"

def get_file_path_from_index(index):
    return path + "Chiquet_" + str(index) + ".csv"

def get_file_paths(n):
    filepaths = []
    for index in range(1, n+1):
        filepaths.append(get_file_path_from_index(index))
    return filepaths

def create_pandaframes(file_paths):
    def discretize_to_0_or_1(row):
        for index in range(len(row)):
            if row[index] != 0:
                row[index] = 1
        return row
    frames = []
    for index in range(0, len(file_paths)):
        frames.append(np.ceil(pd.read_csv(file_paths[index])).astype(int).apply(discretize_to_0_or_1))
    return frames

def check_for_invalid_generation(frames_to_check, extra_frames):
    def check_if_invalid_frame(frame):
        for column in frame.columns:
            if not (frame[column].isin([1]).any()):
                return True
        return False
    
    def find_next_valid_frame():
        not_found_replacement = True
        replacement = extra_frames.pop()
        while(not_found_replacement):
            if not check_if_invalid_frame(replacement):
                break
            replacement = extra_frames.pop()
        return replacement
    for index in range(desired_frames):
        if check_if_invalid_frame(frames[index]):
            print(index)
            replacement = find_next_valid_frame()
            print(replacement)
            replacement.to_csv(get_file_path_from_index(index+1))

In [67]:
frames = create_pandaframes(get_file_paths(total_frames))
check_for_invalid_generation(frames[:desired_frames], frames[desired_frames:])